In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import math
import multiprocessing as mp
import sqlite3

In [2]:
#landscan = gpd.read_file('/Users/kpierce/CooksProTX/texas_avg_pop_30arcsec_with_tract_id/texas_avg_pop_30arcsec_with_tract_id.shp')

Assign an index ID to each value from the landscan raster:

In [3]:
#landscan['pixel_id'] = [i for i in range(landscan.shape[0])]

In [4]:
#landscan.to_file('/Users/kpierce/CooksProTX/texas_avg_pop_30arcsec_with_tract_id/texas_avg_pop_30arcsec_with_tract_id_pixel_id.shp')
landscan = gpd.read_file('/Users/kpierce/CooksProTX/texas_avg_pop_30arcsec_with_tract_id/texas_avg_pop_30arcsec_with_tract_id_pixel_id.shp')

# Census variable interpolation with landscan populations

## Interpolation for persons, housing units, and households

Landscan provides us with a more accurate and more granular understanding of population distribution. The ACS provides us with coarse estimates of population distribution, and counts and percentages of the population falling into certain demographic categories.

We calculate the count and percent of the population in different demographic categories at 30 arcsecond resolution under the following formula:

The total population of a census tract estimated from landscan data is N$_{i}$ = $\sum$N$_{jk}$ for $j$ landscan cells associated with census tract $k$ (1). The population weight (the fraction of the tract population in each landscan cell) is calculated as weight$_{j}$ = N$_{j}$ / N$_{i}$ (2). Following equations 1 and 2, the total number in each demographic category for each landscan cell is calculated as N$_{jm}$ = N$_{km}$ * weight$_{j}$ for demographic category $m$ and census tract $k$ (3). Finally, the percentage of the population in each demographic category in each landscan cell is calculated as percent$_{jm}$ = min(100, N$_{jm}$ / N$_{i}$) (4).

Equation 4 accounts for the possibility that a census-derived population estimate in a demographic category may be larger than a landscan derived population total. These anomalies need further investigation, so the uncorrected percentages, which may range greater than 100%, are included in the outputs.

In some cases, only a single landscan cell centroid will fall within a census tract. We do not consider that some fraction of the landscan cell may be outside of these census tracts in conducting the following interpolations. determined by th

## Dollars

We do not perform interopolation on variables measured in dollars.

# Analysis

## Connect to demographic database

In [5]:
db_name = '/Users/kpierce/protxdb/data/db/cooks_20210923.db'
db_conn = sqlite3.connect(db_name)
db_cursor = db_conn.cursor()

## Solve equation 1

In [6]:
landscan_tract_totals = landscan[['GEOID', 'avg_popula']].groupby(['GEOID']).agg({'avg_popula': sum}).reset_index().rename(columns={'avg_popula': 'landscan_tract_total'})


In [7]:
landscan_tract_totals.head()

,GEOID,landscan_tract_total
0,48001950100,4093.0
1,48001950401,4670.0
2,48001950402,7418.5
3,48001950500,2844.0
4,48001950600,5758.0


## Solve equation 2

In [8]:
landscan_tract_summary = pd.merge(landscan, landscan_tract_totals, on='GEOID', how='left')

In [9]:
landscan_tract_summary.head()

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,pixel_id,geometry,landscan_tract_total
0,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,0,POINT (-97.39167 25.83833),11372.5
1,-97.425000,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,1,POINT (-97.42500 25.84667),11372.5
2,-97.416667,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,2,POINT (-97.41667 25.84667),11372.5
3,-97.408333,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,3,POINT (-97.40833 25.84667),11372.5
4,-97.400000,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,4,POINT (-97.40000 25.84667),11372.5


In [10]:
landscan_tract_summary['population_weight'] = landscan_tract_summary['avg_popula'] / landscan_tract_summary['landscan_tract_total']
landscan_tract_summary_keep = landscan_tract_summary[['x', 'y', 'pixel_id', 'avg_popula', 'GEOID', 'landscan_tract_total', 'population_weight']]

## Solve equation 3

In [11]:
people = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE in ('persons commuting', 'persons') and \
        d.DEMOGRAPHICS_NAME <> 'TOTPOP'
    """, db_conn)

hh = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'households' and \
        d.DEMOGRAPHICS_NAME <> 'HH'
    """, db_conn)

hu = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'housing units' and \
        d.DEMOGRAPHICS_NAME <> 'HU'
    """, db_conn)

dollars = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'dollars'
    """, db_conn)

In [12]:
people_totals = pd.merge(people, landscan_tract_summary_keep, on='GEOID', how='left')
hh_totals = pd.merge(hh, landscan_tract_summary_keep, on='GEOID', how='left')
hu_totals = pd.merge(hu, landscan_tract_summary_keep, on='GEOID', how='left')

all_vars = pd.concat([people_totals, hh_totals, hu_totals])

In [44]:
print(landscan_tract_summary_keep.shape)

(951054, 7)


In [45]:
print(people.dropna().shape)

(173412, 9)


In [46]:
print(hh.dropna().shape)

(5228, 9)


In [47]:
print(hu.dropna().shape)

(15721, 9)


In [13]:
print(people.shape, people_totals.shape)
print(hh.shape, hh_totals.shape)
print(hu.shape, hu_totals.shape)

(173745, 9) (31386960, 15)
(5265, 9) (951120, 15)
(15795, 9) (2853360, 15)


In [14]:
all_vars.head()

,GEOID,GEOTYPE,YEAR,DEMOGRAPHICS_NAME,VALUE,UNITS,NAME,DISPLAY_TEXT,UNIT_OF_MEASURE,x,y,pixel_id,avg_popula,landscan_tract_total,population_weight
0,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.650000,31.871667,589849.0,5.0,4093.0,0.001222
1,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.641667,31.871667,589850.0,1.0,4093.0,0.000244
2,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.633333,31.871667,589851.0,3.5,4093.0,0.000855
3,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.625000,31.871667,589852.0,4.5,4093.0,0.001099
4,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.616667,31.871667,589853.0,0.5,4093.0,0.000122


In [15]:
all_vars['VALUE_COUNT'] = all_vars['VALUE'] * all_vars['population_weight']

## Solve equation 4

In [16]:
all_vars['VALUE_PERCENT_RAW'] = (all_vars['VALUE_COUNT'] / all_vars['landscan_tract_total']) * 100
all_vars['VALUE_PERCENT_CORRECTED'] = [min(i, 100.0) for i in all_vars['VALUE_PERCENT_RAW']]

In [17]:
all_vars.head()

,GEOID,GEOTYPE,YEAR,DEMOGRAPHICS_NAME,VALUE,UNITS,NAME,DISPLAY_TEXT,UNIT_OF_MEASURE,x,y,pixel_id,avg_popula,landscan_tract_total,population_weight,VALUE_COUNT,VALUE_PERCENT_RAW,VALUE_PERCENT_CORRECTED
0,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.650000,31.871667,589849.0,5.0,4093.0,0.001222,1.514781,0.037009,0.037009
1,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.641667,31.871667,589850.0,1.0,4093.0,0.000244,0.302956,0.007402,0.007402
2,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.633333,31.871667,589851.0,3.5,4093.0,0.000855,1.060347,0.025906,0.025906
3,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.625000,31.871667,589852.0,4.5,4093.0,0.001099,1.363303,0.033308,0.033308
4,48001950100,tract,2019,AGE17,1240.0,count,AGE17,population 17 years old and under,persons,-95.616667,31.871667,589853.0,0.5,4093.0,0.000122,0.151478,0.003701,0.003701


## Inspect data

In [18]:
max(all_vars['VALUE_PERCENT_RAW'])

477.09677419354836

In [20]:
excess_pct = all_vars[all_vars['VALUE_PERCENT_RAW'] > 100]

In [21]:
excess_pct.shape

(930, 18)

In [22]:
excess_pct['DEMOGRAPHICS_NAME'].value_counts()

MINRTY                             252
WHITE_ALONE                        241
HISPANIC_LATINO                    145
TOTAL_COMMUTE_POP                  110
WHITE_ALONE_NOT_HISPANIC_LATINO     57
BLACK_AFRICAN_AMERICAN_ALONE        25
FOREIGN_BORN                        23
AGE17                               17
RENTER_OCCUPIED_HU                  16
UNINSUR                             15
POV                                 14
NOHSDP                               4
OTHER_RACE_ALONE                     4
ASIAN_ALONE                          4
LIMENG                               1
GROUPQ                               1
DISABL                               1
Name: DEMOGRAPHICS_NAME, dtype: int64

In [23]:
excess_pct['GEOID'].value_counts()

48113012208    10
48439121906    10
48201423000    10
48453002318     9
48201210700     9
               ..
48201521000     1
48113005100     1
48201432902     1
48113014136     1
48375010100     1
Name: GEOID, Length: 315, dtype: int64

In [24]:
singletons = all_vars[all_vars['population_weight'] == 1.0]

In [25]:
singletons.shape

(10841, 18)

In [26]:
singletons['GEOID'].value_counts()

48085031626    37
48201422702    37
48355001300    37
48113000606    37
48479001201    37
               ..
48141000111    37
48113001002    37
48029131610    37
48201433001    37
48453000803    37
Name: GEOID, Length: 293, dtype: int64

In [27]:
singletons.head()

,GEOID,GEOTYPE,YEAR,DEMOGRAPHICS_NAME,VALUE,UNITS,NAME,DISPLAY_TEXT,UNIT_OF_MEASURE,x,y,pixel_id,avg_popula,landscan_tract_total,population_weight,VALUE_COUNT,VALUE_PERCENT_RAW,VALUE_PERCENT_CORRECTED
43351,48027020701,tract,2019,AGE17,306.0,count,AGE17,population 17 years old and under,persons,-97.341667,31.096667,494757.0,1937.5,1937.5,1.0,306.0,15.793548,15.793548
45932,48029110700,tract,2019,AGE17,259.0,count,AGE17,population 17 years old and under,persons,-98.500000,29.438333,208837.0,1943.0,1943.0,1.0,259.0,13.329902,13.329902
45935,48029110900,tract,2019,AGE17,78.0,count,AGE17,population 17 years old and under,persons,-98.483333,29.438333,208840.0,2969.0,2969.0,1.0,78.0,2.627147,2.627147
46428,48029130200,tract,2019,AGE17,379.0,count,AGE17,population 17 years old and under,persons,-98.466667,29.413333,207829.0,1161.5,1161.5,1.0,379.0,32.630220,32.630220
46429,48029130300,tract,2019,AGE17,1000.0,count,AGE17,population 17 years old and under,persons,-98.466667,29.405000,207722.0,1047.0,1047.0,1.0,1000.0,95.510984,95.510984


In [28]:
landscan[landscan['GEOID'] == '48029131610']

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,pixel_id,geometry
211561,-98.316667,29.496667,1519.5,1809,48,029,131610,48029131610,1316.10,Census Tract 1316.10,G5020,S,1826199.0,0.0,+29.4949003,-098.3180134,211561,POINT (-98.31667 29.49667)


In [30]:
n_obs = all_vars[['pixel_id', 'DEMOGRAPHICS_NAME']].value_counts()
max(n_obs)

1

We still have only a single observation per landscan cell -- no accidental fan-outs on merge.

# Reformat data and save

In [31]:
all_vars.to_csv('/Users/kpierce/CooksProTX/landscan_people_hh_hu_vars_30arcsec.csv')

The NaNs come from the original demographic data where all tracts are represented in the SQLite file, but not all tracts have values for all variables.

In [41]:
all_vars_complete = all_vars.dropna()

In [43]:
print(all_vars.shape)
print(all_vars_complete.shape)

(35191440, 18)
(35055357, 18)


In [48]:
all_vars_count = all_vars_complete[['pixel_id', 'x', 'y', 'GEOID', 'DEMOGRAPHICS_NAME', 'VALUE_COUNT']]

all_vars_count_wide = all_vars_count.pivot(
    index=['pixel_id', 'x', 'y', 'GEOID'],
    columns='DEMOGRAPHICS_NAME',
    values='VALUE_COUNT'
)

all_vars_count_wide.columns = ['E_' + i for i in all_vars_count_wide.columns]

In [53]:
all_vars_count_wide.head()

,,,,E_10_14_MIN,E_15_19_MIN,E_20_24_MIN,E_25_29_MIN,E_30_34_MIN,E_35_39_MIN,E_40_44_MIN,E_45_59_MIN,E_5LESS_MIN,E_5_9_MIN,...,E_NOVEH,E_OTHER_RACE_ALONE,E_POV,E_RENTER_OCCUPIED_HU,E_TOTAL_COMMUTE_POP,E_TWO_OR_MORE_RACES,E_UNEMP,E_UNINSUR,E_WHITE_ALONE,E_WHITE_ALONE_NOT_HISPANIC_LATINO
pixel_id,x,y,GEOID,,,,,,,,,,,,,,,,,,,,,
0.0,-97.391667,25.838333,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,-97.425000,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,-97.416667,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,-97.408333,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,-97.400000,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
all_vars_percent = all_vars_complete[['pixel_id', 'x', 'y', 'GEOID', 'DEMOGRAPHICS_NAME', 'VALUE_PERCENT_CORRECTED']]

all_vars_percent_wide = all_vars_percent.pivot(
    index=['pixel_id', 'x', 'y', 'GEOID'],
    columns='DEMOGRAPHICS_NAME',
    values='VALUE_PERCENT_CORRECTED'
)

all_vars_percent_wide.columns = ['EP_' + i for i in all_vars_percent_wide.columns]

In [56]:
all_vars_percent_wide.head()

,,,,EP_10_14_MIN,EP_15_19_MIN,EP_20_24_MIN,EP_25_29_MIN,EP_30_34_MIN,EP_35_39_MIN,EP_40_44_MIN,EP_45_59_MIN,EP_5LESS_MIN,EP_5_9_MIN,...,EP_NOVEH,EP_OTHER_RACE_ALONE,EP_POV,EP_RENTER_OCCUPIED_HU,EP_TOTAL_COMMUTE_POP,EP_TWO_OR_MORE_RACES,EP_UNEMP,EP_UNINSUR,EP_WHITE_ALONE,EP_WHITE_ALONE_NOT_HISPANIC_LATINO
pixel_id,x,y,GEOID,,,,,,,,,,,,,,,,,,,,,
0.0,-97.391667,25.838333,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,-97.425000,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,-97.416667,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,-97.408333,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,-97.400000,25.846667,48061014100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
dollars_ls = .head()

,GEOID,GEOTYPE,YEAR,DEMOGRAPHICS_NAME,VALUE,UNITS,NAME,DISPLAY_TEXT,UNIT_OF_MEASURE
0,48001950100,tract,2019,PCI,27923.0,count,PCI,per capita income,dollars
1,48001950401,tract,2019,PCI,2357.0,count,PCI,per capita income,dollars
2,48001950402,tract,2019,PCI,1039.0,count,PCI,per capita income,dollars
3,48001950500,tract,2019,PCI,17100.0,count,PCI,per capita income,dollars
4,48001950600,tract,2019,PCI,22549.0,count,PCI,per capita income,dollars


In [77]:
dollars = dollars.dropna()
dollars_ls = pd.merge(dollars, landscan_tract_summary_keep, on='GEOID', how='left')
dollars_keep = dollars_ls[['pixel_id', 'x', 'y', 'GEOID', 'DEMOGRAPHICS_NAME', 'VALUE']]
dollars_wide = dollars_keep.pivot(
    index=['pixel_id', 'x', 'y', 'GEOID'],
    columns='DEMOGRAPHICS_NAME',
    values='VALUE'
)

dollars_wide = dollars_wide.dropna()

In [79]:
m1 = pd.merge(all_vars_percent_wide, all_vars_count_wide, left_index=True, right_index=True)
final = pd.merge(m1, dollars_wide, left_index=True, right_index=True)

In [80]:
final.shape

(930929, 76)

In [81]:
final.to_csv('/Users/kpierce/CooksProTX/landscan_people_hh_hu_income_vars_30arcsec_20210923.csv')

This output is provisional; the landscan merge has a few problems. Some tracts are missing:

In [70]:
ls_tract_list = landscan['GEOID'].unique()
demo_tract_list = set(list(people['GEOID'].unique()) + list(hh['GEOID'].unique()) + list(hu['GEOID'].unique()))

In [72]:
len(demo_tract_list.difference(ls_tract_list))

66

In [73]:
len(set(ls_tract_list).difference(demo_tract_list))

0

In [76]:
print(len(ls_tract_list))
print(len(demo_tract_list))
print(1-(len(ls_tract_list)/len(demo_tract_list)))

5199
5265
0.012535612535612528
